# Impact Score

### Baseline

In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler
import torch
import torch.nn as nn
from sentence_transformers import SentenceTransformer
from collections import defaultdict
from sklearn.metrics import r2_score

c:\Users\user\anaconda3\envs\tensor\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import pandas as pd

In [4]:
# 1. 데이터 준비 및 타겟 생성
df1 = pd.read_csv("../../db/news_2023_2025_external.csv")
df2 = pd.read_csv("../../db/news_2023_2025_metadata.csv")

df = df1.merge(df2, on="news_id")
df.replace([np.inf, -np.inf], np.nan, inplace=True)

df = df.dropna()

NameError: name 'np' is not defined

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 13425 entries, 238 to 13748
Data columns (total 38 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   news_id                     13425 non-null  object 
 1   d_minus_5_date_close        13425 non-null  float64
 2   d_minus_5_date_volume       13425 non-null  float64
 3   d_minus_5_date_foreign      13425 non-null  float64
 4   d_minus_5_date_institution  13425 non-null  float64
 5   d_minus_5_date_individual   13425 non-null  float64
 6   d_minus_4_date_close        13425 non-null  float64
 7   d_minus_4_date_volume       13425 non-null  float64
 8   d_minus_4_date_foreign      13425 non-null  float64
 9   d_minus_4_date_institution  13425 non-null  float64
 10  d_minus_4_date_individual   13425 non-null  float64
 11  d_minus_3_date_close        13425 non-null  float64
 12  d_minus_3_date_volume       13425 non-null  float64
 13  d_minus_3_date_foreign      13425 

In [5]:
# 2. 타겟 생성
target_cols = ['d_plus_1_date_close', 'd_plus_2_date_close', 'd_plus_3_date_close', 'd_plus_4_date_close', 'd_plus_5_date_close']
df['target'] = df[target_cols].values.tolist()

In [8]:
train_targets = df[target_cols].values.tolist()

In [10]:
import numpy as np

train_targets = df[target_cols].values.tolist()  # → list of lists
train_targets = np.array(train_targets)  # → (N, 5) numpy array

baseline_mean = train_targets.mean(axis=0)  # shape: (5,)
baseline_std = train_targets.std(axis=0)  # shape: (5,)

In [11]:
baseline_mean

array([-0.0078573 , -0.0083497 , -0.00810895, -0.00799404, -0.00891119])

In [12]:
baseline_std

array([0.06588189, 0.07283064, 0.07710478, 0.08238087, 0.08863377])

특성공학컬럼 생성

In [ ]:
def feature_engineer_stock_data(df: pd.DataFrame) -> pd.DataFrame:
    # 관련 숫자형 컬럼들을 float으로 변환
    numeric_cols_to_convert = [col for col in df.columns if 'd_minus' in col and ('close' in col or 'volume' in col or 'foreign' in col or 'institution' in col or 'individual' in col) and df[col].dtype == 'int64']
    for col in numeric_cols_to_convert:
        df.loc[:, col] = df[col].astype(float)

    # 'news_id'에서 'date' 컬럼 생성
    if 'news_id' in df.columns:
        try:
            df.loc[:, 'date'] = pd.to_datetime(df['news_id'].str[:8], format='%Y%m%d', errors='coerce')
        except Exception:
            df.loc[:, 'date'] = pd.NaT
    else:
        df.loc[:, 'date'] = pd.NaT

    # 'stock_list'에서 'stock_code' 컬럼 생성
    if 'stock_list' in df.columns:
        df.loc[:, 'stock_code'] = df['stock_list'].apply(
            lambda x: x[0] if isinstance(x, list) and len(x) > 0 else np.nan
        )
        df.loc[:, 'stock_code'] = df['stock_code'].fillna('Unknown_Stock_For_Imputation')
    else:
        df.loc[:, 'stock_code'] = 'Unknown_Stock_For_Imputation'

    # 1.1. 일별 수익률 특성 추가
    daily_return_cols = []
    for i in range(1, 5):
        current_day_close = f'd_minus_{i}_date_close'
        prev_day_close = f'd_minus_{i+1}_date_close'
        new_col_name = f'd_minus_{i}_daily_return'

        if current_day_close in df.columns and prev_day_close in df.columns:
            df.loc[:, new_col_name] = (df[current_day_close] - df[prev_day_close]) / df[prev_day_close].replace(0, np.nan)
            daily_return_cols.append(new_col_name)

    # 1.2. 이동 평균 특성 추가
    moving_average_cols = []
    if all(col in df.columns for col in ['d_minus_1_date_close', 'd_minus_2_date_close', 'd_minus_3_date_close']):
        df.loc[:, 'ma_3_d_minus_1'] = df[['d_minus_1_date_close', 'd_minus_2_date_close', 'd_minus_3_date_close']].mean(axis=1)
        moving_average_cols.append('ma_3_d_minus_1')

    if all(col in df.columns for col in ['d_minus_1_date_close', 'd_minus_2_date_close', 'd_minus_3_date_close', 'd_minus_4_date_close', 'd_minus_5_date_close']):
        df.loc[:, 'ma_5_d_minus_1'] = df[['d_minus_1_date_close', 'd_minus_2_date_close', 'd_minus_3_date_close', 'd_minus_4_date_close', 'd_minus_5_date_close']].mean(axis=1)
        moving_average_cols.append('ma_5_d_minus_1')

    # 1.3. 변동성 특성 추가
    volatility_cols = []
    if daily_return_cols and len(daily_return_cols) >= 2:
        df.loc[:, 'vol_5_d_minus_1'] = df[daily_return_cols].std(axis=1)
        volatility_cols.append('vol_5_d_minus_1')

    # 특성 이상치 및 결측치(NaN) 처리 
    features_to_process_and_impute = daily_return_cols + moving_average_cols + volatility_cols

    if not features_to_process_and_impute:
        return df

    # 퀀타일 계산을 위한 임시 데이터프레임 생성
    temp_df_for_quantile = df[features_to_process_and_impute].dropna()

    for col in daily_return_cols:
        if col in temp_df_for_quantile.columns and not temp_df_for_quantile[col].empty:
            lower_bound = temp_df_for_quantile[col].quantile(0.01)
            upper_bound = temp_df_for_quantile[col].quantile(0.99)
            df.loc[:, col] = np.clip(df[col], lower_bound, upper_bound)

    if 'vol_5_d_minus_1' in df.columns and 'vol_5_d_minus_1' in temp_df_for_quantile.columns and not temp_df_for_quantile['vol_5_d_minus_1'].empty:
        upper_bound_vol = temp_df_for_quantile['vol_5_d_minus_1'].quantile(0.99)
        df.loc[:, 'vol_5_d_minus_1'] = np.clip(df['vol_5_d_minus_1'], 0, upper_bound_vol)

    # NaN 값 처리: 그룹별 interpolate()를 사용하여 결측치 채우기
    if 'stock_code' in df.columns and 'date' in df.columns and not df['date'].isnull().all():
        df_sorted = df.sort_values(by=['stock_code', 'date']).copy()

        for col in features_to_process_and_impute:
            if col in df_sorted.columns:
                df_sorted.loc[:, col] = df_sorted.groupby('stock_code')[col].transform(
                    lambda x: x.interpolate(method='linear', limit_direction='both', limit_area='inside')
                )
                df_sorted.loc[:, col] = df_sorted.groupby('stock_code')[col].transform(
                    lambda x: x.fillna(method='ffill').fillna(method='bfill')
                )
                if df_sorted[col].isna().any():
                    df_sorted.loc[:, col] = df_sorted[col].fillna(0)

        df = df_sorted.loc[df.index].copy()
    else:
        for col in features_to_process_and_impute:
            if col in df.columns and df[col].isna().any():
                fill_value = df[col].mean()
                if not pd.isna(fill_value):
                    df.loc[:, col] = df[col].fillna(fill_value)
                else:
                    df.loc[:, col] = df[col].fillna(0)
    return df

df = feature_engineer_stock_data(df.copy())

C:\Users\user\AppData\Local\Temp\ipykernel_10688\3736084660.py:21: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['Unknown_Stock_For_Imputation' 'Unknown_Stock_For_Imputation'
 'Unknown_Stock_For_Imputation' ... 'Unknown_Stock_For_Imputation'
 'Unknown_Stock_For_Imputation' 'Unknown_Stock_For_Imputation']' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[:, 'stock_code'] = df['stock_code'].fillna('Unknown_Stock_For_Imputation')
C:\Users\user\AppData\Local\Temp\ipykernel_10688\3736084660.py:81: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  lambda x: x.fillna(method='ffill').fillna(method='bfill')
C:\Users\user\AppData\Local\Temp\ipykernel_10688\3736084660.py:81: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfil

피쳐 스케일링

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 13425 entries, 238 to 13748
Data columns (total 48 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   news_id                     13425 non-null  object        
 1   d_minus_5_date_close        13425 non-null  float64       
 2   d_minus_5_date_volume       13425 non-null  float64       
 3   d_minus_5_date_foreign      13425 non-null  float64       
 4   d_minus_5_date_institution  13425 non-null  float64       
 5   d_minus_5_date_individual   13425 non-null  float64       
 6   d_minus_4_date_close        13425 non-null  float64       
 7   d_minus_4_date_volume       13425 non-null  float64       
 8   d_minus_4_date_foreign      13425 non-null  float64       
 9   d_minus_4_date_institution  13425 non-null  float64       
 10  d_minus_4_date_individual   13425 non-null  float64       
 11  d_minus_3_date_close        13425 non-null  float64      

In [ ]:
# standard
group_price_close = [
    'd_minus_5_date_close', 'd_minus_4_date_close',
    'd_minus_3_date_close', 'd_minus_2_date_close',
    'd_minus_1_date_close'
]

# RobustScaler
group_volume = [
    'd_minus_5_date_volume', 'd_minus_4_date_volume',
    'd_minus_3_date_volume', 'd_minus_2_date_volume',
    'd_minus_1_date_volume'
]

# MinMaxScaler
group_foreign = [
    'd_minus_5_date_foreign', 'd_minus_4_date_foreign',
    'd_minus_3_date_foreign', 'd_minus_2_date_foreign',
    'd_minus_1_date_foreign'
]

# MinMaxScaler
group_institution = [
    'd_minus_5_date_institution', 'd_minus_4_date_institution',
    'd_minus_3_date_institution', 'd_minus_2_date_institution',
    'd_minus_1_date_institution'
]

# MinMaxScaler
group_individual = [
    'd_minus_5_date_individual', 'd_minus_4_date_individual',
    'd_minus_3_date_individual', 'd_minus_2_date_individual',
    'd_minus_1_date_individual'
]

# StandardScaler
group_macro = ['fx', 'bond10y', 'base_rate']

# RobustScaler
daily_return_cols = ['d_minus_1_daily_return', 'd_minus_2_daily_return', 'd_minus_3_daily_return', 'd_minus_4_daily_return']

# StandardScaler
moving_average_cols = ['ma_3_d_minus_1', 'ma_5_d_minus_1']

# RobustScaler
volatility_cols = ['vol_5_d_minus_1']


groups = [
    group_macro,
    group_volume,
    group_foreign,
    group_institution,
    group_individual,
    group_price_close,     # 기존 종가 데이터
    daily_return_cols,     # 수익률 특성
    moving_average_cols,   # 이동 평균 특성
    volatility_cols        # 변동성 특성
]

scaler_choices = [
    StandardScaler(), # group_macro
    RobustScaler(),   # group_volume
    MinMaxScaler(),   # group_foreign
    MinMaxScaler(),   # group_institution
    MinMaxScaler(),   # group_individual
    StandardScaler(), # group_price_close 
    RobustScaler(),   # daily_return_cols
    StandardScaler(), # moving_average_cols 
    RobustScaler()    # volatility_cols
]

external_cols = []
for group in groups:
    # 그룹 내 컬럼이 실제로 DataFrame에 존재하는지 다시 확인 
    valid_group_cols = [col for col in group if col in df.columns]
    external_cols.extend(valid_group_cols)


# 3. 훈련/검증 분할 및 스케일링 
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)

fitted_scalers = {}

print("\n--- 훈련/검증 데이터 스케일링 시작 ---")
for i, group in enumerate(groups):
    scaler = scaler_choices[i]
    # 해당 그룹의 모든 컬럼이 train_df에 있는지 확인하고 유효한 컬럼만 사용
    group_in_train = [col for col in group if col in train_df.columns]
    group_in_val = [col for col in group if col in val_df.columns]

    if not group_in_train:
        continue # 이 그룹 스케일링 건너뛰기

    scaler.fit(train_df[group_in_train])
    fitted_scalers[i] = scaler

    train_df.loc[:, group_in_train] = scaler.transform(train_df[group_in_train])
    if group_in_val: # 검증 데이터에도 해당 컬럼이 있을 경우에만 변환
        val_df.loc[:, group_in_val] = scaler.transform(val_df[group_in_val])
    else:
        pass


# 타겟 변수 스케일링 
target_scaler = StandardScaler()
train_targets_np = np.array(train_df['target'].tolist())
val_targets_np = np.array(val_df['target'].tolist())

target_scaler.fit(train_targets_np)

train_df['target_scaled'] = target_scaler.transform(train_targets_np).tolist()
val_df['target_scaled'] = target_scaler.transform(val_targets_np).tolist()



최종 external_cols 수: 35
최종 external_cols 예시: ['fx', 'bond10y', 'base_rate', 'd_minus_5_date_volume', 'd_minus_4_date_volume', 'd_minus_3_date_volume', 'd_minus_2_date_volume', 'd_minus_1_date_volume', 'd_minus_5_date_foreign', 'd_minus_4_date_foreign', 'd_minus_3_date_foreign', 'd_minus_2_date_foreign', 'd_minus_1_date_foreign', 'd_minus_5_date_institution', 'd_minus_4_date_institution', 'd_minus_3_date_institution', 'd_minus_2_date_institution', 'd_minus_1_date_institution', 'd_minus_5_date_individual', 'd_minus_4_date_individual', 'd_minus_3_date_individual', 'd_minus_2_date_individual', 'd_minus_1_date_individual', 'd_minus_5_date_close', 'd_minus_4_date_close', 'd_minus_3_date_close', 'd_minus_2_date_close', 'd_minus_1_date_close', 'd_minus_1_daily_return', 'd_minus_2_daily_return', 'd_minus_3_daily_return', 'd_minus_4_daily_return', 'ma_3_d_minus_1', 'ma_5_d_minus_1', 'vol_5_d_minus_1'] ...

--- 훈련/검증 데이터 스케일링 시작 ---


문장 임베딩 생성

In [12]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# 임베딩 모델 초기화
model_emb = SentenceTransformer(
    "snunlp/KR-SBERT-V40K-klueNLI-augSTS",
    device='cuda' if torch.cuda.is_available() else 'cpu'
)

print(f"\n▶ 훈련 데이터 임베딩 생성 ({len(train_df)}개 뉴스)")
train_summary_embedding = model_emb.encode(
    train_df["summary"].tolist(),
    batch_size=32,
    show_progress_bar=True,
    convert_to_tensor=True
)
train_df["embedding"] = train_summary_embedding.cpu().numpy().tolist()

print(f"\n▶ 검증 데이터 임베딩 생성 ({len(val_df)}개 뉴스)")
val_summary_embedding = model_emb.encode(
    val_df["summary"].tolist(),
    batch_size=32,
    show_progress_bar=True,
    convert_to_tensor=True
)
val_df["embedding"] = val_summary_embedding.cpu().numpy().tolist()

c:\Users\user\anaconda3\envs\tensor\lib\site-packages\huggingface_hub\file_download.py:144: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\user\.cache\huggingface\hub\models--snunlp--KR-SBERT-V40K-klueNLI-augSTS. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)



▶ 훈련 데이터 임베딩 생성 (10740개 뉴스)


Batches: 100%|██████████| 336/336 [00:26<00:00, 12.81it/s]



▶ 검증 데이터 임베딩 생성 (2685개 뉴스)


Batches: 100%|██████████| 84/84 [00:06<00:00, 13.43it/s]


데이터셋

In [13]:
# 5. Dataset 및 DataLoader
from torch.utils.data import Dataset, DataLoader

class CustomDataset(Dataset):
    def __init__(self, df, external_cols):
        self.embeddings = torch.tensor(df["embedding"].tolist(), dtype=torch.float32)
        self.external_features = torch.tensor(df[external_cols].values, dtype=torch.float32)
        self.targets = torch.tensor(df["target_scaled"].tolist(), dtype=torch.float32)

    def __len__(self):
        return len(self.embeddings)

    def __getitem__(self, idx):
        return self.embeddings[idx], self.external_features[idx], self.targets[idx]

train_dataset = CustomDataset(train_df, external_cols)
val_dataset = CustomDataset(val_df, external_cols)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

오토인코더

In [14]:
class Autoencoder(nn.Module):
    def __init__(self, input_dim, latent_dim):
        super().__init__()
        # self.bert는 이 AE 모델의 입력으로 직접 사용되지 않음.
        # 외부에서 생성된 임베딩을 받는 구조로 변경
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, latent_dim * 2),
            nn.GELU(),
            nn.Linear(latent_dim * 2, latent_dim)
        )
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, latent_dim * 2),
            nn.GELU(),
            nn.Linear(latent_dim * 2, input_dim)
        )

    def forward(self, x):
        latent = self.encoder(x)
        reconstructed = self.decoder(latent)
        return reconstructed, latent

# 임베딩 차원 (SentenceTransformer 출력 차원)
embedding_dim = model_emb.get_sentence_embedding_dimension()
latent_dim = 128 

ae_model = Autoencoder(input_dim=embedding_dim, latent_dim=latent_dim).to(device)

def freeze_module(module):
    for param in module.parameters():
        param.requires_grad = False

def train_autoencoder(model, train_loader, val_loader=None, epochs=10, device='cuda'):
    ae_optimizer = torch.optim.AdamW(
        list(model.encoder.parameters()) + list(model.decoder.parameters()),
        lr=1e-4,
        weight_decay=1e-5
    )
    criterion = nn.MSELoss()

    model.encoder.train()
    model.decoder.train()

    print("\n▶ 오토인코더 사전학습 시작")
    for epoch in range(epochs):
        total_loss = 0
        for batch_embedding, _, _ in train_loader: # DataLoader에서 임베딩만 사용
            embeddings = batch_embedding.to(device)

            latent = model.encoder(embeddings)
            reconstructed = model.decoder(latent)

            loss = criterion(reconstructed, embeddings)
            ae_optimizer.zero_grad()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            ae_optimizer.step()

            total_loss += loss.item()

        val_loss = 0
        if val_loader:
            model.eval()
            with torch.no_grad():
                for val_batch_embedding, _, _ in val_loader: # DataLoader에서 임베딩만 사용
                    val_embeddings = val_batch_embedding.to(device)
                    val_latent = model.encoder(val_embeddings)
                    val_reconstructed = model.decoder(val_latent)
                    val_loss += criterion(val_reconstructed, val_embeddings).item()
            model.train() # 다시 학습 모드로 전환
            print(f"[AE] Epoch {epoch+1} | Train Loss: {total_loss/len(train_loader):.4f} | Val Loss: {val_loss/len(val_loader):.4f}")
        else:
            print(f"[AE] Epoch {epoch+1} | Train Loss: {total_loss/len(train_loader):.4f}")

    model.encoder.eval()
    model.decoder.eval()
    return model

# 오토인코더 학습 (여기서는 임베딩만 사용)
ae_model = train_autoencoder(ae_model, train_loader, val_loader, epochs=10, device=device)


▶ 오토인코더 사전학습 시작
[AE] Epoch 1 | Train Loss: 0.1634 | Val Loss: 0.1152
[AE] Epoch 2 | Train Loss: 0.0975 | Val Loss: 0.0833
[AE] Epoch 3 | Train Loss: 0.0759 | Val Loss: 0.0692
[AE] Epoch 4 | Train Loss: 0.0648 | Val Loss: 0.0607
[AE] Epoch 5 | Train Loss: 0.0577 | Val Loss: 0.0550
[AE] Epoch 6 | Train Loss: 0.0526 | Val Loss: 0.0506
[AE] Epoch 7 | Train Loss: 0.0486 | Val Loss: 0.0472
[AE] Epoch 8 | Train Loss: 0.0455 | Val Loss: 0.0444
[AE] Epoch 9 | Train Loss: 0.0429 | Val Loss: 0.0420
[AE] Epoch 10 | Train Loss: 0.0407 | Val Loss: 0.0401


예측 모델 재구성

In [15]:
class ImportancePredictor(nn.Module):
    def __init__(self, ae_encoder, external_feature_dim, fcl_hidden=1024, dropout=0.25):  # hidden 크기 조금 키워도 좋아용
        super().__init__()
        self.encoder = ae_encoder
        encoder_output_dim = ae_encoder[-1].out_features if isinstance(ae_encoder[-1], nn.Linear) else latent_dim

        self.predictor = nn.Sequential(
            nn.Linear(encoder_output_dim + external_feature_dim, fcl_hidden),
            nn.GELU(),
            nn.BatchNorm1d(fcl_hidden),
            nn.Dropout(dropout),

            nn.Linear(fcl_hidden, fcl_hidden // 2),
            nn.GELU(),
            nn.BatchNorm1d(fcl_hidden // 2),
            nn.Dropout(dropout),

            nn.Linear(fcl_hidden // 2, fcl_hidden // 4),
            nn.GELU(),
            nn.BatchNorm1d(fcl_hidden // 4),
            nn.Dropout(dropout),

            nn.Linear(fcl_hidden // 4, fcl_hidden // 8), 
            nn.GELU(),
            nn.BatchNorm1d(fcl_hidden // 8),
            nn.Dropout(dropout),

            nn.Linear(fcl_hidden // 8, 5)
        )

    def forward(self, embedding, external):
        latent = self.encoder(embedding)
        combined = torch.cat([latent, external], dim=1)
        return self.predictor(combined)

# ImportancePredictor 초기화
predictor = ImportancePredictor(
    ae_encoder=ae_model.encoder,
    external_feature_dim=len(external_cols) 
).to(device)

학습 및 검증 (프리징)

In [16]:
def train_predictor(
    model,
    train_loader,
    val_loader,
    target_scaler,
    device='cuda',
    epochs=150, # 최대 에포크 증가
    patience=15, # 조기 종료 patience 증가
    max_grad_norm=1.0
):
    # 1. 프리징 초기화 (한 번만 실행)
    freeze_module(model.encoder)  # 인코더 영구 고정
    model.encoder.eval()  # BN/Dropout 비활성화

    # 2. 옵티마이저 설정
    optimizer = torch.optim.AdamW(
        model.predictor.parameters(),  # 예측기만 등록
        lr=5e-6,
        weight_decay=1e-6
    )
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer,
        'min',
        patience=5, # 스케줄러 patience 조정
        factor=0.5
    )
    criterion = nn.HuberLoss() # HuberLoss 유지

    best_val_loss = float('inf')
    patience_counter = 0

    history = defaultdict(list)

    scaler = torch.cuda.amp.GradScaler()

    print("\n▶ 메인 예측기 학습 시작")
    for epoch in range(epochs):
        # --------- 훈련 단계 ---------
        model.train()

        train_loss = 0
        for batch_embedding, batch_external, batch_targets in train_loader:
            embeddings = batch_embedding.to(device)
            external_features = batch_external.to(device)
            targets = batch_targets.to(device)

            optimizer.zero_grad()

            with torch.autocast(device_type=device, dtype=torch.float16):
                preds = model(embeddings, external_features)
                loss = criterion(preds, targets)

            scaler.scale(loss).backward()
            torch.nn.utils.clip_grad_norm_(
                model.predictor.parameters(),  # 예측기만 클리핑
                max_grad_norm
            )
            scaler.step(optimizer)
            scaler.update()

            train_loss += loss.item()

        avg_train_loss = train_loss / len(train_loader)
        history['train_loss'].append(avg_train_loss)

        # --------- 검증 단계 ---------
        model.eval()
        total_val_loss = 0
        all_preds, all_targets = [], []
        with torch.no_grad():
            for batch_embedding, batch_external, batch_targets in val_loader:
                embeddings = batch_embedding.to(device)
                external_features = batch_external.to(device)
                targets = batch_targets.to(device)

                with torch.autocast(device_type=device, dtype=torch.float16):
                    preds = model(embeddings, external_features)
                    loss = criterion(preds, targets)
                    total_val_loss += loss.item()
                    all_preds.append(preds.cpu())
                    all_targets.append(targets.cpu())

        avg_val_loss = total_val_loss / len(val_loader)
        history['val_loss'].append(avg_val_loss)

        all_preds_np = torch.cat(all_preds).numpy()
        all_targets_np = torch.cat(all_targets).numpy()
        preds_inv = target_scaler.inverse_transform(all_preds_np)
        targets_inv = target_scaler.inverse_transform(all_targets_np)

        mae = nn.L1Loss()(torch.tensor(preds_inv), torch.tensor(targets_inv)).item()
        rmse = torch.sqrt(nn.MSELoss()(torch.tensor(preds_inv), torch.tensor(targets_inv))).item()
        r2 = r2_score(targets_inv, preds_inv)

        print(f"[Epoch {epoch+1}] Train Loss: {avg_train_loss:.4f} | "
              f"Val Loss: {avg_val_loss:.4f} | MAE: {mae:.2f} | RMSE: {rmse:.2f} | R²: {r2:.4f} | "
              f"LR: {optimizer.param_groups[0]['lr']:.2e}")

        direction_match = (np.sign(preds_inv) == np.sign(targets_inv))
        directional_accuracy = np.mean(direction_match)
        print(f"Directional Accuracy: {directional_accuracy:.4f}")


        scheduler.step(avg_val_loss)

        # --------- 조기 종료 ---------
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            patience_counter = 0
            best_epoch = epoch + 1
            best_mae = mae
            best_r2 = r2
        else:
            patience_counter += 1
            print(f" -> 조기 종료 카운터: {patience_counter}/{patience}")
            if patience_counter >= patience:
                print(f" -> epoch {epoch+1}에서 조기 종료!")
                break

    print(f"최적 모델: Epoch {best_epoch} | MAE: {best_mae:.2f} | R²: {best_r2:.4f}")
    return model

# 최종 예측기 학습 실행
trained_predictor = train_predictor(
    predictor,
    train_loader,
    val_loader,
    target_scaler,
    device=device,
    epochs=150, # 최대 에포크 증가
    patience=15 # 조기 종료 patience 증가
)

C:\Users\user\AppData\Local\Temp\ipykernel_10688\2944849852.py:34: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler()



▶ 메인 예측기 학습 시작
[Epoch 1] Train Loss: 0.4712 | Val Loss: 0.3768 | MAE: 0.05 | RMSE: 0.09 | R²: -0.0887 | LR: 5.00e-06
Directional Accuracy: 0.4898
[Epoch 2] Train Loss: 0.4608 | Val Loss: 0.3690 | MAE: 0.05 | RMSE: 0.08 | R²: -0.0712 | LR: 5.00e-06
Directional Accuracy: 0.4924
[Epoch 3] Train Loss: 0.4498 | Val Loss: 0.3637 | MAE: 0.05 | RMSE: 0.08 | R²: -0.0570 | LR: 5.00e-06
Directional Accuracy: 0.4956
[Epoch 4] Train Loss: 0.4416 | Val Loss: 0.3597 | MAE: 0.05 | RMSE: 0.08 | R²: -0.0480 | LR: 5.00e-06
Directional Accuracy: 0.5038
[Epoch 5] Train Loss: 0.4349 | Val Loss: 0.3578 | MAE: 0.05 | RMSE: 0.08 | R²: -0.0429 | LR: 5.00e-06
Directional Accuracy: 0.5004
[Epoch 6] Train Loss: 0.4297 | Val Loss: 0.3535 | MAE: 0.05 | RMSE: 0.08 | R²: -0.0319 | LR: 5.00e-06
Directional Accuracy: 0.5032
[Epoch 7] Train Loss: 0.4253 | Val Loss: 0.3528 | MAE: 0.05 | RMSE: 0.08 | R²: -0.0327 | LR: 5.00e-06
Directional Accuracy: 0.5061
[Epoch 8] Train Loss: 0.4238 | Val Loss: 0.3530 | MAE: 0.05 | RMSE:

모델 저장

In [17]:
import joblib
import torch.onnx

# 입력 특성별 스케일러 저장
for idx, scaler in fitted_scalers.items():
    joblib.dump(scaler, f"scaler_group_{idx}.joblib")

# 타겟 스케일러 저장
joblib.dump(target_scaler, "target_scaler.joblib")


# 임베딩 차원 정보
dummy_embedding = torch.randn(1, embedding_dim, device=device)

# encoder만 ONNX로 저장
torch.onnx.export(
    ae_model.encoder,
    dummy_embedding,
    "ae_encoder.onnx",
    input_names=["embedding"],
    output_names=["latent"],
    dynamic_axes={"embedding": {0: "batch_size"}, "latent": {0: "batch_size"}},
    opset_version=17
)

# 외부 피처 차원 정보
external_feature_dim = len(external_cols)
dummy_external = torch.randn(1, external_feature_dim, device=device)

# predictor 전체 ONNX로 저장
torch.onnx.export(
    predictor,
    (dummy_embedding, dummy_external),
    "predictor.onnx",
    input_names=["embedding", "external"],
    output_names=["output"],
    dynamic_axes={
        "embedding": {0: "batch_size"},
        "external": {0: "batch_size"},
        "output": {0: "batch_size"}
    },
    opset_version=17
)


In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 13425 entries, 238 to 13748
Data columns (total 48 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   news_id                     13425 non-null  object        
 1   d_minus_5_date_close        13425 non-null  float64       
 2   d_minus_5_date_volume       13425 non-null  float64       
 3   d_minus_5_date_foreign      13425 non-null  float64       
 4   d_minus_5_date_institution  13425 non-null  float64       
 5   d_minus_5_date_individual   13425 non-null  float64       
 6   d_minus_4_date_close        13425 non-null  float64       
 7   d_minus_4_date_volume       13425 non-null  float64       
 8   d_minus_4_date_foreign      13425 non-null  float64       
 9   d_minus_4_date_institution  13425 non-null  float64       
 10  d_minus_4_date_individual   13425 non-null  float64       
 11  d_minus_3_date_close        13425 non-null  float64      

In [20]:
df.target

238           [0.01, 0.03, 0.03, 0.03, 0.05]
239      [-0.05, -0.03, -0.09, -0.02, -0.05]
240         [0.02, 0.03, 0.02, -0.01, -0.01]
241        [-0.0, 0.03, -0.01, -0.04, -0.04]
242           [0.01, 0.04, 0.03, 0.01, 0.02]
                        ...                 
13744         [0.02, 0.03, 0.05, 0.05, 0.05]
13745    [-0.01, -0.03, -0.04, -0.05, -0.06]
13746     [-0.4, -0.33, -0.33, -0.38, -0.49]
13747         [0.02, 0.03, 0.05, 0.05, 0.05]
13748         [0.03, 0.03, 0.04, 0.03, 0.06]
Name: target, Length: 13425, dtype: object

----------------------------------------------------------

In [19]:
import torch
import pickle
import os
from transformers import AutoTokenizer

# 모델 저장
output_dir = './saved_models'
os.makedirs(output_dir, exist_ok=True)
torch.save(trained_predictor.state_dict(), './saved_models/model_weights.pth')

# 1. 타겟 스케일러 (target_scaler) 저장
with open(os.path.join(output_dir, 'target_scaler.pkl'), 'wb') as f:
    pickle.dump(target_scaler, f)

# 2. 외부 특성 스케일러들 (fitted_scalers) 저장
with open(os.path.join(output_dir, 'external_fitted_scalers.pkl'), 'wb') as f:
    pickle.dump(fitted_scalers, f)

# # 토크나이저 저장
# trained_predictor.tokenizer.save_pretrained('./saved_models')  # transformers 토크나이저라면
# # 이러면 vocab.txt, tokenizer_config.json, special_tokens_map.json 등이 저장됨



In [22]:
import torch
import torch.nn as nn
import pickle
import os
from sentence_transformers import SentenceTransformer

# 1. 모델 클래스 재정의 (기존과 동일)
class Autoencoder(nn.Module):
    def __init__(self, input_dim, latent_dim):
        super().__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, latent_dim * 2),
            nn.GELU(),
            nn.Linear(latent_dim * 2, latent_dim)
        )
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, latent_dim * 2),
            nn.GELU(),
            nn.Linear(latent_dim * 2, input_dim)
        )

    def forward(self, x):
        latent = self.encoder(x)
        reconstructed = self.decoder(latent)
        return reconstructed, latent

class ImportancePredictor(nn.Module):
    def __init__(self, ae_encoder, external_feature_dim, fcl_hidden=1024, dropout=0.25):
        super().__init__()
        self.encoder = ae_encoder
        encoder_output_dim = ae_encoder[-1].out_features if isinstance(ae_encoder[-1], nn.Linear) else 128

        self.predictor = nn.Sequential(
            nn.Linear(encoder_output_dim + external_feature_dim, fcl_hidden),
            nn.GELU(),
            nn.BatchNorm1d(fcl_hidden),
            nn.Dropout(dropout),

            nn.Linear(fcl_hidden, fcl_hidden // 2),
            nn.GELU(),
            nn.BatchNorm1d(fcl_hidden // 2),
            nn.Dropout(dropout),

            nn.Linear(fcl_hidden // 2, fcl_hidden // 4),
            nn.GELU(),
            nn.BatchNorm1d(fcl_hidden // 4),
            nn.Dropout(dropout),

            nn.Linear(fcl_hidden // 4, fcl_hidden // 8),
            nn.GELU(),
            nn.BatchNorm1d(fcl_hidden // 8),
            nn.Dropout(dropout),

            nn.Linear(fcl_hidden // 8, 5)
        )

    def forward(self, embedding, external):
        latent = self.encoder(embedding)
        combined = torch.cat([latent, external], dim=1)
        return self.predictor(combined)

# 2. 모델 설정값 정의 (학습 시와 동일하게)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model_emb = SentenceTransformer("snunlp/KR-SBERT-V40K-klueNLI-augSTS")
embedding_dim = model_emb.get_sentence_embedding_dimension()  # 768
latent_dim = 128

# external_cols 정의 (학습 시와 동일하게)
external_cols = [
    # 매크로 경제
    'fx', 'bond10y', 'base_rate',
    # 거래량
    'd_minus_5_date_volume', 'd_minus_4_date_volume', 'd_minus_3_date_volume',
    'd_minus_2_date_volume', 'd_minus_1_date_volume',
    # 외국인
    'd_minus_5_date_foreign', 'd_minus_4_date_foreign', 'd_minus_3_date_foreign',
    'd_minus_2_date_foreign', 'd_minus_1_date_foreign',
    # 기관
    'd_minus_5_date_institution', 'd_minus_4_date_institution', 'd_minus_3_date_institution',
    'd_minus_2_date_institution', 'd_minus_1_date_institution',
    # 개인
    'd_minus_5_date_individual', 'd_minus_4_date_individual', 'd_minus_3_date_individual',
    'd_minus_2_date_individual', 'd_minus_1_date_individual',
    # 종가
    'd_minus_5_date_close', 'd_minus_4_date_close', 'd_minus_3_date_close',
    'd_minus_2_date_close', 'd_minus_1_date_close',
    # 수익률 (실제 데이터에 따라 조정 필요)
    'd_minus_1_daily_return', 'd_minus_2_daily_return', 'd_minus_3_daily_return', 'd_minus_4_daily_return',
    # 이동평균
    'ma_3_d_minus_1', 'ma_5_d_minus_1',
    # 변동성
    'vol_5_d_minus_1'
]

# 3. 모델 구조 재생성
ae_model = Autoencoder(input_dim=embedding_dim, latent_dim=latent_dim).to(device)
loaded_predictor = ImportancePredictor(
    ae_encoder=ae_model.encoder,
    external_feature_dim=len(external_cols)
).to(device)

# 4. 저장된 가중치 로드
model_path = './saved_models/model_weights.pth'
loaded_predictor.load_state_dict(torch.load(model_path, map_location=device))
loaded_predictor.eval()

# 5. 스케일러들 로드
with open('./saved_models/target_scaler.pkl', 'rb') as f:
    target_scaler = pickle.load(f)

with open('./saved_models/external_fitted_scalers.pkl', 'rb') as f:
    fitted_scalers = pickle.load(f)

print("모델 및 스케일러 로드 완료!")

# 6. ONNX 변환
dummy_embedding = torch.randn(1, embedding_dim).to(device)
dummy_external = torch.randn(1, len(external_cols)).to(device)

# 모델을 evaluation 모드로 설정
loaded_predictor.eval()

torch.onnx.export(
    loaded_predictor,
    (dummy_embedding, dummy_external),
    "importance_predictor.onnx",
    input_names=["embedding", "external"],
    output_names=["prediction"],
    dynamic_axes={
        "embedding": {0: "batch_size"},
        "external": {0: "batch_size"},
        "prediction": {0: "batch_size"}
    },
    opset_version=17,
    do_constant_folding=True,
    export_params=True
)

print("ONNX 변환 완료: importance_predictor.onnx")


모델 및 스케일러 로드 완료!
ONNX 변환 완료: importance_predictor.onnx


사용예시

In [58]:
# 1. 데이터 준비 및 타겟 생성
df1 = pd.read_csv("news_2023_2025_external.csv")
df2 = pd.read_csv("news_2023_2025_metadata.csv")

df2 = df1.merge(df2, on="news_id")
df2.replace([np.inf, -np.inf], np.nan, inplace=True)
df2 = df2.dropna()

In [47]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 13425 entries, 238 to 13748
Data columns (total 38 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   news_id                     13425 non-null  object 
 1   d_minus_5_date_close        13425 non-null  float64
 2   d_minus_5_date_volume       13425 non-null  float64
 3   d_minus_5_date_foreign      13425 non-null  float64
 4   d_minus_5_date_institution  13425 non-null  float64
 5   d_minus_5_date_individual   13425 non-null  float64
 6   d_minus_4_date_close        13425 non-null  float64
 7   d_minus_4_date_volume       13425 non-null  float64
 8   d_minus_4_date_foreign      13425 non-null  float64
 9   d_minus_4_date_institution  13425 non-null  float64
 10  d_minus_4_date_individual   13425 non-null  float64
 11  d_minus_3_date_close        13425 non-null  float64
 12  d_minus_3_date_volume       13425 non-null  float64
 13  d_minus_3_date_foreign      13425 

In [56]:
import numpy as np
import onnxruntime as ort
import pickle
import pandas as pd
from sentence_transformers import SentenceTransformer

# 1. ONNX 모델 로드
sess = ort.InferenceSession(
    "importance_predictor.onnx",
    providers=['CPUExecutionProvider']  # CUDA 없으니 CPU만 사용
)

# 2. 스케일러들 로드
with open('./saved_models/target_scaler.pkl', 'rb') as f:
    target_scaler = pickle.load(f)

with open('./saved_models/external_fitted_scalers.pkl', 'rb') as f:
    fitted_scalers = pickle.load(f)

# 3. 임베딩 모델 로드
model_emb = SentenceTransformer("snunlp/KR-SBERT-V40K-klueNLI-augSTS")

# 4. 특성공학 함수 (학습 코드와 동일하게 구현)
def create_engineered_features_complete(df):
    """학습 시와 정확히 동일한 특성공학"""
    df = df.copy()

    print("특성공학 시작...")
    print(f"특성공학 전 컬럼 수: {len(df.columns)}")

    # 1. 일별 수익률 계산
    daily_return_cols = []
    for i in range(1, 5):
        current_close_key = f'd_minus_{i}_date_close'
        prev_close_key = f'd_minus_{i+1}_date_close'
        return_key = f'd_minus_{i}_daily_return'

        if current_close_key in df.columns and prev_close_key in df.columns:
            df[return_key] = (df[current_close_key] - df[prev_close_key]) / df[prev_close_key].replace(0, np.nan)
            df[return_key] = df[return_key].fillna(0)
            daily_return_cols.append(return_key)
            print(f"  생성: {return_key}")

    # 2. 이동평균 계산
    moving_average_cols = []
    if all(col in df.columns for col in ['d_minus_1_date_close', 'd_minus_2_date_close', 'd_minus_3_date_close']):
        df['ma_3_d_minus_1'] = df[['d_minus_1_date_close', 'd_minus_2_date_close', 'd_minus_3_date_close']].mean(axis=1)
        moving_average_cols.append('ma_3_d_minus_1')
        print(f"  생성: ma_3_d_minus_1")

    if all(col in df.columns for col in ['d_minus_1_date_close', 'd_minus_2_date_close', 'd_minus_3_date_close', 'd_minus_4_date_close', 'd_minus_5_date_close']):
        df['ma_5_d_minus_1'] = df[['d_minus_1_date_close', 'd_minus_2_date_close', 'd_minus_3_date_close', 'd_minus_4_date_close', 'd_minus_5_date_close']].mean(axis=1)
        moving_average_cols.append('ma_5_d_minus_1')
        print(f"  생성: ma_5_d_minus_1")

    # 3. 변동성 계산
    volatility_cols = []
    if len(daily_return_cols) >= 2:
        df['vol_5_d_minus_1'] = df[daily_return_cols].std(axis=1)
        df['vol_5_d_minus_1'] = df['vol_5_d_minus_1'].fillna(0)
        volatility_cols.append('vol_5_d_minus_1')
        print(f"  생성: vol_5_d_minus_1")

    print(f"특성공학 후 컬럼 수: {len(df.columns)}")
    print(f"새로 생성된 컬럼: {daily_return_cols + moving_average_cols + volatility_cols}")

    return df

def get_complete_external_cols():
    """학습 시와 정확히 동일한 external_cols 순서"""
    # 학습 코드와 정확히 동일한 그룹 순서
    group_macro = ['fx', 'bond10y', 'base_rate']  # 3개
    group_volume = ['d_minus_5_date_volume', 'd_minus_4_date_volume', 'd_minus_3_date_volume', 'd_minus_2_date_volume', 'd_minus_1_date_volume']  # 5개
    group_foreign = ['d_minus_5_date_foreign', 'd_minus_4_date_foreign', 'd_minus_3_date_foreign', 'd_minus_2_date_foreign', 'd_minus_1_date_foreign']  # 5개
    group_institution = ['d_minus_5_date_institution', 'd_minus_4_date_institution', 'd_minus_3_date_institution', 'd_minus_2_date_institution', 'd_minus_1_date_institution']  # 5개
    group_individual = ['d_minus_5_date_individual', 'd_minus_4_date_individual', 'd_minus_3_date_individual', 'd_minus_2_date_individual', 'd_minus_1_date_individual']  # 5개
    group_price_close = ['d_minus_5_date_close', 'd_minus_4_date_close', 'd_minus_3_date_close', 'd_minus_2_date_close', 'd_minus_1_date_close']  # 5개
    daily_return_cols = ['d_minus_1_daily_return', 'd_minus_2_daily_return', 'd_minus_3_daily_return', 'd_minus_4_daily_return']  # 4개
    moving_average_cols = ['ma_3_d_minus_1', 'ma_5_d_minus_1']  # 2개
    volatility_cols = ['vol_5_d_minus_1']  # 1개

    # 학습 시와 동일한 순서
    groups = [
        group_macro,           # 0
        group_volume,          # 1
        group_foreign,         # 2
        group_institution,     # 3
        group_individual,      # 4
        group_price_close,     # 5
        daily_return_cols,     # 6
        moving_average_cols,   # 7
        volatility_cols        # 8
    ]

    external_cols = []
    for group in groups:
        external_cols.extend(group)

    return external_cols, groups

def apply_scaling_complete(df, fitted_scalers):
    """학습 시와 정확히 동일한 스케일링"""
    df_scaled = df.copy()
    external_cols, groups = get_complete_external_cols()

    print(f"스케일링 시작... 기대 컬럼 수: {len(external_cols)}")

    for group_idx, group_cols in enumerate(groups):
        if group_idx in fitted_scalers:
            scaler = fitted_scalers[group_idx]
            valid_cols = [col for col in group_cols if col in df.columns]
            missing_cols = [col for col in group_cols if col not in df.columns]

            if missing_cols:
                print(f"그룹 {group_idx} 누락 컬럼: {missing_cols}")

            if valid_cols:
                df_scaled[valid_cols] = scaler.transform(df[valid_cols])
                print(f"그룹 {group_idx}: {len(valid_cols)}/{len(group_cols)}개 컬럼 스케일링 완료")
            else:
                print(f"그룹 {group_idx}: 유효한 컬럼 없음")

    return df_scaled, external_cols

def debug_onnx_complete():
    print("=== 완전한 ONNX 디버깅 시작 ===")

    # 데이터 로드
    sample_df = df2.iloc[4567:4577].copy()
    print(f"샘플 데이터 크기: {sample_df.shape}")
    print(f"샘플 인덱스: {sample_df.index.tolist()}")

    # 1. 특성공학 적용 (누락된 파생 특성들 생성)
    print("\n=== 특성공학 적용 ===")
    sample_df_engineered = create_engineered_features_complete(sample_df)

    # 2. 스케일링 적용
    print("\n=== 스케일링 적용 ===")
    sample_df_scaled, external_cols = apply_scaling_complete(sample_df_engineered, fitted_scalers)

    print(f"최종 external_cols 수: {len(external_cols)}")
    print(f"external_cols: {external_cols}")

    # 3. 첫 번째 샘플로 디버깅
    idx = sample_df_scaled.index[0]
    row = sample_df_scaled.loc[idx]

    print(f"\n=== 샘플 {idx} 디버깅 ===")
    print(f"뉴스: {row['summary'][:100]}...")

    # 임베딩 생성
    embedding = model_emb.encode([row['summary']])
    embedding = embedding.astype(np.float32)
    print(f"임베딩 형태: {embedding.shape}")

    # 외부 특성 벡터 생성 (35개 컬럼 모두 포함)
    external_vector = []
    missing_cols = []
    for col in external_cols:
        if col in row.index:
            external_vector.append(row[col])
        else:
            external_vector.append(0.0)  # 누락된 컬럼은 0으로 채움
            missing_cols.append(col)

    if missing_cols:
        print(f"누락된 컬럼들 (0으로 채움): {missing_cols}")

    external_array = np.array([external_vector], dtype=np.float32)
    print(f"외부 특성 형태: {external_array.shape}")
    print(f"외부 특성 값 범위: {external_array.min():.6f} ~ {external_array.max():.6f}")

    # ONNX 추론 실행
    inputs = {"embedding": embedding, "external": external_array}

    print(f"\n=== ONNX 추론 실행 ===")
    try:
        raw_predictions = sess.run(None, inputs)[0]

        print(f"ONNX 원시 출력: {raw_predictions}")
        print(f"각 값별 상세: {[f'{x:.8f}' for x in raw_predictions[0]]}")

        # 타겟 스케일러 역변환
        predictions_original = target_scaler.inverse_transform(raw_predictions)
        print(f"역변환 후 출력: {predictions_original}")
        print(f"역변환 후 상세: {[f'{x:.8f}' for x in predictions_original[0]]}")

        # 실제 값과 비교
        actual_values = [
            sample_df.loc[idx, 'd_plus_1_date_close'],
            sample_df.loc[idx, 'd_plus_2_date_close'],
            sample_df.loc[idx, 'd_plus_3_date_close'],
            sample_df.loc[idx, 'd_plus_4_date_close'],
            sample_df.loc[idx, 'd_plus_5_date_close']
        ]
        print(f"실제 값: {actual_values}")

        print("\n=== 예측 결과 vs 실제 값 ===")
        for i in range(5):
            print(f"D+{i+1} - 예측: {predictions_original[0][i]:.6f}, 실제: {actual_values[i]:.6f}")

    except Exception as e:
        print(f"ONNX 추론 중 오류: {e}")
        import traceback
        traceback.print_exc()

    print("\n=== 디버깅 완료 ===")

# 실행
debug_onnx_complete()


=== 완전한 ONNX 디버깅 시작 ===
샘플 데이터 크기: (10, 38)
샘플 인덱스: [4838, 4839, 4840, 4841, 4842, 4843, 4844, 4845, 4846, 4847]

=== 특성공학 적용 ===
특성공학 시작...
특성공학 전 컬럼 수: 38
  생성: d_minus_1_daily_return
  생성: d_minus_2_daily_return
  생성: d_minus_3_daily_return
  생성: d_minus_4_daily_return
  생성: ma_3_d_minus_1
  생성: ma_5_d_minus_1
  생성: vol_5_d_minus_1
특성공학 후 컬럼 수: 45
새로 생성된 컬럼: ['d_minus_1_daily_return', 'd_minus_2_daily_return', 'd_minus_3_daily_return', 'd_minus_4_daily_return', 'ma_3_d_minus_1', 'ma_5_d_minus_1', 'vol_5_d_minus_1']

=== 스케일링 적용 ===
스케일링 시작... 기대 컬럼 수: 35
그룹 0: 3/3개 컬럼 스케일링 완료
그룹 1: 5/5개 컬럼 스케일링 완료
그룹 2: 5/5개 컬럼 스케일링 완료
그룹 3: 5/5개 컬럼 스케일링 완료
그룹 4: 5/5개 컬럼 스케일링 완료
그룹 5: 5/5개 컬럼 스케일링 완료
그룹 6: 4/4개 컬럼 스케일링 완료
그룹 7: 2/2개 컬럼 스케일링 완료
그룹 8: 1/1개 컬럼 스케일링 완료
최종 external_cols 수: 35
external_cols: ['fx', 'bond10y', 'base_rate', 'd_minus_5_date_volume', 'd_minus_4_date_volume', 'd_minus_3_date_volume', 'd_minus_2_date_volume', 'd_minus_1_date_volume', 'd_minus_5_date_foreign', 'd_minus_4_date_fo

최종 이용코드

In [59]:
import numpy as np
import onnxruntime as ort
import pickle
import pandas as pd
from sentence_transformers import SentenceTransformer

class NewsImpactPredictor:
    def __init__(self):
        """ONNX 모델과 스케일러들을 한 번에 로드"""
        print("=== 모델 초기화 중 ===")

        # ONNX 모델 로드
        self.sess = ort.InferenceSession(
            "importance_predictor.onnx",
            providers=['CPUExecutionProvider']
        )

        # 스케일러들 로드
        with open('./saved_models/target_scaler.pkl', 'rb') as f:
            self.target_scaler = pickle.load(f)

        with open('./saved_models/external_fitted_scalers.pkl', 'rb') as f:
            self.fitted_scalers = pickle.load(f)

        # 임베딩 모델 로드
        self.model_emb = SentenceTransformer("snunlp/KR-SBERT-V40K-klueNLI-augSTS")

        print("모델 초기화 완료!")

    def create_engineered_features(self, df):
        """특성공학 적용"""
        df = df.copy()

        # 1. 일별 수익률 계산
        for i in range(1, 5):
            current_close_key = f'd_minus_{i}_date_close'
            prev_close_key = f'd_minus_{i+1}_date_close'
            return_key = f'd_minus_{i}_daily_return'

            if current_close_key in df.columns and prev_close_key in df.columns:
                df[return_key] = (df[current_close_key] - df[prev_close_key]) / df[prev_close_key].replace(0, np.nan)
                df[return_key] = df[return_key].fillna(0)

        # 2. 이동평균 계산
        if all(col in df.columns for col in ['d_minus_1_date_close', 'd_minus_2_date_close', 'd_minus_3_date_close']):
            df['ma_3_d_minus_1'] = df[['d_minus_1_date_close', 'd_minus_2_date_close', 'd_minus_3_date_close']].mean(axis=1)

        if all(col in df.columns for col in ['d_minus_1_date_close', 'd_minus_2_date_close', 'd_minus_3_date_close', 'd_minus_4_date_close', 'd_minus_5_date_close']):
            df['ma_5_d_minus_1'] = df[['d_minus_1_date_close', 'd_minus_2_date_close', 'd_minus_3_date_close', 'd_minus_4_date_close', 'd_minus_5_date_close']].mean(axis=1)

        # 3. 변동성 계산
        return_cols = [f'd_minus_{i}_daily_return' for i in range(1, 5)]
        existing_return_cols = [col for col in return_cols if col in df.columns]
        if len(existing_return_cols) >= 2:
            df['vol_5_d_minus_1'] = df[existing_return_cols].std(axis=1)
            df['vol_5_d_minus_1'] = df['vol_5_d_minus_1'].fillna(0)

        return df

    def apply_scaling(self, df):
        """스케일링 적용"""
        df_scaled = df.copy()

        # 학습 시와 동일한 그룹 정의
        group_macro = ['fx', 'bond10y', 'base_rate']
        group_volume = ['d_minus_5_date_volume', 'd_minus_4_date_volume', 'd_minus_3_date_volume', 'd_minus_2_date_volume', 'd_minus_1_date_volume']
        group_foreign = ['d_minus_5_date_foreign', 'd_minus_4_date_foreign', 'd_minus_3_date_foreign', 'd_minus_2_date_foreign', 'd_minus_1_date_foreign']
        group_institution = ['d_minus_5_date_institution', 'd_minus_4_date_institution', 'd_minus_3_date_institution', 'd_minus_2_date_institution', 'd_minus_1_date_institution']
        group_individual = ['d_minus_5_date_individual', 'd_minus_4_date_individual', 'd_minus_3_date_individual', 'd_minus_2_date_individual', 'd_minus_1_date_individual']
        group_price_close = ['d_minus_5_date_close', 'd_minus_4_date_close', 'd_minus_3_date_close', 'd_minus_2_date_close', 'd_minus_1_date_close']
        daily_return_cols = ['d_minus_1_daily_return', 'd_minus_2_daily_return', 'd_minus_3_daily_return', 'd_minus_4_daily_return']
        moving_average_cols = ['ma_3_d_minus_1', 'ma_5_d_minus_1']
        volatility_cols = ['vol_5_d_minus_1']

        groups = [
            group_macro,           # 0 - StandardScaler
            group_volume,          # 1 - RobustScaler
            group_foreign,         # 2 - MinMaxScaler
            group_institution,     # 3 - MinMaxScaler
            group_individual,      # 4 - MinMaxScaler
            group_price_close,     # 5 - StandardScaler
            daily_return_cols,     # 6 - RobustScaler
            moving_average_cols,   # 7 - StandardScaler
            volatility_cols        # 8 - RobustScaler
        ]

        for group_idx, group_cols in enumerate(groups):
            if group_idx in self.fitted_scalers:
                scaler = self.fitted_scalers[group_idx]
                valid_cols = [col for col in group_cols if col in df.columns]

                if valid_cols:
                    df_scaled[valid_cols] = scaler.transform(df[valid_cols])

        return df_scaled, groups

    def predict_impact_score(self, df_row):
        """단일 행에 대해 impact_score 예측"""
        # DataFrame으로 변환 (단일 행 처리를 위해)
        if isinstance(df_row, pd.Series):
            df_single = df_row.to_frame().T
        else:
            df_single = df_row.copy()

        # 1. 특성공학 적용
        df_engineered = self.create_engineered_features(df_single)

        # 2. 스케일링 적용
        df_scaled, groups = self.apply_scaling(df_engineered)

        # 3. external_cols 생성
        external_cols = []
        for group in groups:
            for col in group:
                if col in df_scaled.columns:
                    external_cols.append(col)

        # 4. 뉴스 임베딩 생성
        news_text = df_scaled.iloc[0]['summary']
        embedding = self.model_emb.encode([news_text])
        embedding = embedding.astype(np.float32)

        # 5. 외부 특성 벡터 생성
        row = df_scaled.iloc[0]
        external_vector = []
        for col in external_cols:
            if col in row.index:
                external_vector.append(row[col])
            else:
                external_vector.append(0.0)

        external_array = np.array([external_vector], dtype=np.float32)

        # 6. ONNX 추론 실행
        inputs = {"embedding": embedding, "external": external_array}
        raw_predictions = self.sess.run(None, inputs)[0]

        # 7. 역변환
        predictions_original = self.target_scaler.inverse_transform(raw_predictions)

        # 8. impact_score 계산 (5일 평균 변화율의 절댓값)
        impact_score = np.mean(np.abs(predictions_original[0]))

        return impact_score

    def update_impact_score(self, df, index=None):
        """
        DataFrame의 특정 행 또는 전체 행에 대해 impact_score 업데이트

        Args:
            df (DataFrame): 업데이트할 DataFrame
            index (int/list/None): 업데이트할 행 인덱스. None이면 전체 업데이트

        Returns:
            DataFrame: impact_score가 업데이트된 DataFrame
        """
        df_updated = df.copy()

        # impact_score 컬럼이 없으면 생성
        if 'impact_score' not in df_updated.columns:
            df_updated['impact_score'] = 0.0

        # 업데이트할 인덱스 결정
        if index is None:
            indices_to_update = df_updated.index
        elif isinstance(index, (int, np.integer)):
            indices_to_update = [index]
        else:
            indices_to_update = index

        print(f"=== {len(indices_to_update)}개 행의 impact_score 업데이트 중 ===")

        for i, idx in enumerate(indices_to_update):
            try:
                row = df_updated.loc[idx]
                impact_score = self.predict_impact_score(row)
                df_updated.loc[idx, 'impact_score'] = impact_score

                print(f"[{i+1}/{len(indices_to_update)}] 인덱스 {idx}: impact_score = {impact_score:.6f}")

            except Exception as e:
                print(f"인덱스 {idx} 처리 중 오류: {e}")
                df_updated.loc[idx, 'impact_score'] = 0.0

        print("=== impact_score 업데이트 완료 ===")
        return df_updated


In [ ]:
# # 사용법
# def quick_update_impact_score(df, index=None):
#     """
#     빠른 impact_score 업데이트 함수

#     사용 예시:
#     - 전체 업데이트: df = quick_update_impact_score(df)
#     - 특정 행 업데이트: df = quick_update_impact_score(df, 100)
#     - 여러 행 업데이트: df = quick_update_impact_score(df, [100, 101, 102])
#     """
#     global _predictor

#     # 전역 predictor가 없으면 생성
#     if '_predictor' not in globals():
#         _predictor = NewsImpactPredictor()

#     return _predictor.update_impact_score(df, index)

# # 사용 예시들
# if __name__ == "__main__":
#     # 1. 전체 DataFrame 업데이트
#     print("=== 사용 예시 1: 전체 DataFrame 업데이트 ===")
#     # df_updated = quick_update_impact_score(df2)

#     # 2. 특정 행 하나만 업데이트
#     print("=== 사용 예시 2: 특정 행 업데이트 ===")
#     # df_updated = quick_update_impact_score(df2, 4838)

#     # 3. 여러 행 업데이트
#     print("=== 사용 예시 3: 여러 행 업데이트 ===")
#     df_updated = quick_update_impact_score(df2.iloc[4567:4577], None)  # 10개 행만

#     print("\n업데이트된 결과:")
#     print(df_updated[['news_id', 'summary', 'impact_score']].head())


In [61]:
import numpy as np
import onnxruntime as ort
import pickle
import pandas as pd
from sentence_transformers import SentenceTransformer
import random

# 랜덤 행 선택 및 실행
def demo_random_prediction():
    print("=== df2 랜덤 행 impact_score 예측 데모 ===\n")

    # 1. 예측기 초기화
    predictor = NewsImpactPredictor()

    # 2. df2에서 랜덤 행 선택
    random_index = random.choice(df2.index)
    print(f"선택된 랜덤 인덱스: {random_index}")

    # 3. 선택된 행 정보 출력
    selected_row = df2.loc[random_index]
    print(f"뉴스 ID: {selected_row['news_id']}")
    print(f"뉴스 요약: {selected_row['summary'][:150]}...")
    print(f"주식 리스트: {selected_row['stock_list']}")
    print(f"현재 impact_score: {selected_row.get('impact_score', 'N/A')}")

    # 4. 실제 미래 종가 정보
    actual_values = [
        selected_row['d_plus_1_date_close'],
        selected_row['d_plus_2_date_close'],
        selected_row['d_plus_3_date_close'],
        selected_row['d_plus_4_date_close'],
        selected_row['d_plus_5_date_close']
    ]
    print(f"\n실제 미래 5일 변화율: {[f'{x:.4f}' for x in actual_values]}")

    # 5. impact_score 예측
    print(f"\n=== 예측 수행 중 ===")
    try:
        impact_score, predictions = predictor.predict_impact_score(selected_row)

        print(f"\n=== 예측 결과 ===")
        print(f"예측된 impact_score: {impact_score:.6f}")
        print(f"예측된 5일 변화율: {[f'{x:.4f}' for x in predictions]}")

        print(f"\n=== 실제 vs 예측 비교 ===")
        for i in range(5):
            diff = abs(predictions[i] - actual_values[i])
            print(f"D+{i+1}: 예측 {predictions[i]:.4f} vs 실제 {actual_values[i]:.4f} (차이: {diff:.4f})")

        # 6. 방향성 정확도 계산
        pred_signs = np.sign(predictions)
        actual_signs = np.sign(actual_values)
        direction_match = np.sum(pred_signs == actual_signs)
        print(f"\n방향성 정확도: {direction_match}/5 ({direction_match/5*100:.1f}%)")

        # 7. df2 업데이트 (옵션)
        print(f"\n=== DataFrame 업데이트 ===")
        df2.loc[random_index, 'impact_score'] = impact_score
        print(f"인덱스 {random_index}의 impact_score가 {impact_score:.6f}로 업데이트되었습니다!")

        # 8. 업데이트된 정보 확인
        updated_row = df2.loc[random_index]
        print(f"업데이트된 impact_score: {updated_row['impact_score']:.6f}")

    except Exception as e:
        print(f"예측 중 오류 발생: {e}")
        import traceback
        traceback.print_exc()

# 실행
demo_random_prediction()

=== df2 랜덤 행 impact_score 예측 데모 ===

=== 모델 초기화 중 ===
모델 초기화 완료!
선택된 랜덤 인덱스: 10701
뉴스 ID: 20240418_0100
뉴스 요약: 한미반도체는 주주가치 제고를 위해 이달 26일까지 발행주식의 0.36%에 해당하는 34만5668주를 소각한다고 공시했으며 전날 종가 기준으로 470억원 규모의 자사주 소각 대상 주식은 한미반도체가 지난해 10월17일 300억원 규모 자사주 신탁계약 체결을 통해 취득한 ...
주식 리스트: ["한미반도체"]
현재 impact_score: 0.4750190391253094

실제 미래 5일 변화율: ['-0.0000', '0.0700', '0.0400', '-0.0500', '-0.0000']

=== 예측 수행 중 ===
예측 중 오류 발생: cannot unpack non-iterable numpy.float32 object


<ipython-input-59-4233191722>:42: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[return_key] = df[return_key].fillna(0)
<ipython-input-59-4233191722>:42: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[return_key] = df[return_key].fillna(0)
<ipython-input-59-4233191722>:42: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[return_key] =